In [9]:
import optparse, os, glob, codecs, time, re

In [10]:
import bs4

In [11]:
def choisir_fichier(default=u"."):
    for file in glob.glob(default):
        print file
    return raw_input(u"Fichier Talismane : ")

In [12]:
def monter(origin,numero):
    gouv=mot[numero]["gouv"]
    phrase[gouv]["nums"].append(mot[origin]["num"])
    if gouv!=0:
        monter(origin,gouv)

In [13]:
def baliser(chaine,balise,attvals=[]):
    if attvals:
        result='<%s'%balise
        for element in attvals:
            if "=" in element:
                (attr,val)=element.split("=")
                result=result+' %s="%s"'%(attr,val)
            else:
                if element!="_":
                    result=result+' type="%s"'%(element)
        result=result+'>%s</%s>' % (chaine, balise)
    else:
        result='<%s>%s</%s>' % (balise, chaine, balise)
    return result

In [14]:
def baliser_groupe(liste,debut,fin,balise,attvals=[]):
    if attvals:
        result='<%s'%balise
        for element in attvals:
            if "=" in element:
                (attr,val)=element.split("=")
                result=result+' %s="%s"'%(attr.replace("+","_plus_"),val.replace("+","_plus_"))
            else:
                if element!="_":
                    result=result+' type="%s"'%(element.replace("+","_plus_"))
                    
        result=result+'>'
        liste[debut]=result+liste[debut]
        liste[fin]=liste[fin]+'</%s>'%(balise.replace("+","_plus_"))
    else:
        liste[debut]='<%s>'%(balise)+liste[debut]
        liste[fin]=liste[fin]+'</%s>'%(balise.replace("+","_plus_"))

In [15]:
xml_lines=[]
tag_file_name=choisir_fichier(u"*.tal")

Eclypsia3.tal
phraseJean.tal
Fichier Talismane : DM3/Louna3ansCharles.tal


In [16]:
with codecs.open(tag_file_name.strip(),"r","utf-8") as tag_file:
    tag_data=tag_file.readlines()
file_name, sep, after = tag_file_name.rpartition(".")
output_file_name=file_name+"-"+time.strftime("%y%m%d")+".xml"

In [17]:
xml_lines.append('<?xml version="1.0" encoding="UTF-8"?>')
xml_lines.append('<!--?xml-stylesheet type="text/css" href="affichage.css"?-->')

In [18]:
mots=[(0,"",[])]
mot={0:{"num":0, "forme":"", "cat":"PHR"}}
phrase={0:{"nums":[],"forme":"", "lemme":"", "cat":"PHR","gouv":0, "rel":""}}

In [19]:
for line in tag_data:
    line = line.strip()
    if "\t" in line:
        (num, forme, lemme, cat, catdet, morsyn, gouv, rel, trail1,trail2)=line.split("\t")
        morsyn = morsyn.strip("|")
        morsyns=morsyn.split("|")
        if "_" in forme:
            temp=forme.split("_")
            forme=" ".join(temp)
        if lemme == "_": lemme = forme
        mot[int(num)]={"num":int(num), "forme":forme, "lemme":lemme, "cat":cat, "gouv":int(gouv), "rel":rel, "morsyns":morsyns}
        phrase[int(num)]={"nums":[int(num)], "forme":forme, "lemme":lemme, "cat":cat, "gouv":int(gouv), "rel":rel}
    elif re.match(r"<[^?].*>", line):
        # print line
        xml_lines.append(line)
    else:
        positions=[i for i in range(len(mot))]
        suite=""
        for i in range(1,len(mot)):
            monter(i,i)
            L = ["num=%s"%i,"lexeme="+mot[i]["lemme"].upper(),"cat="+mot[i]["cat"].replace("+","_plus_")]
            mots.append((i,baliser(mot[i]["forme"],"mot", mot[i]["morsyns"].extend(L)),phrase[i]["nums"]))
            positions[i]=baliser(mot[i]["forme"],"mot",mot[i]["morsyns"])  #mot[i]["cat"].replace("+","_plus_")
            if mot[i]["forme"] in ",." or suite.endswith("'"):
                suite=suite+mot[i]["forme"]
            else:
                suite=suite+" "+mot[i]["forme"]
        # print mot, xml_mots
        suite = re.sub(r"\(-+ *\)", "(-)", suite)
        #xml_lines.append('<!-- %s -->'%suite)
        xml_mots=mots[:]
        xml_mots.sort(key=lambda x: len(x[2]))
        for element in xml_mots:
            element[2].sort()
            if element[2]:
                relation="fonction=%s"%phrase[element[0]]["rel"].replace("+","_plus_")
                syntagme="G%s"%phrase[element[0]]["cat"].replace("+","_plus_")
                num_tete="num_tete=%s"%element[0]
                gouverneur="cat_gouv=%s"%mot[phrase[element[0]]["gouv"]]["cat"].replace("+","_plus_")
                num_gouverneur="num_gouv=%s"%mot[phrase[element[0]]["gouv"]]["num"]
                baliser_groupe(positions,int(element[2][:1][0]),int(element[2][-1:][0]),syntagme,[relation,num_tete,gouverneur,num_gouverneur])
        mots.sort(key=lambda x: x[0])
        for position in positions:
            if position!=0:
                xml_lines.append(position)
        xml_lines.append("")
        xml_lines.append("")      
        mots=[(0,"",[])]
        mot={0:{"num":0, "forme":"", "cat":"PHR"}}
        phrase={0:{"nums":[],"forme":"", "lemme":"", "cat":"PHR","gouv":0, "rel":""}}

In [20]:
with codecs.open(output_file_name, "w", "utf-8") as output_file:
    xml_text="\n".join(xml_lines)
    soup=bs4.BeautifulSoup(xml_text,"xml")
    output_file.write(soup.prettify())